### Importando e estruturando os dados

Fonte dos dados: https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao

In [1]:
import pandas as pd

# lendo o arquivo baixado no Open Data SUS
dados = pd.read_csv("vacinacao_alagoas_09_09_2021.csv", sep=";", encoding='utf-8') 

In [2]:
dados.head(1)

,document_id,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,...,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem
0,7353e614-eb78-4181-9777-e8f0c8209cbd-i0b0,4fd8495e2e26869a224fa4ae2c132fee8727f3a13df29c...,57.0,1964-04-22,M,2.0,PRETA,270550.0,10.0,MURICI,...,210216,FUNDACAO BUTANTAN,Organization/61189445000156,2021-05-17,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-05-18T12:42:16.000Z,16341


### Filtrar apenas algumas colunas do DataFrame (planilha)

In [22]:
df = dados[['vacina_nome','vacina_dataaplicacao', 'vacina_descricao_dose']]
df.head(5)

,vacina_nome,vacina_dataaplicacao,vacina_descricao_dose
0,Covid-19-Coronavac-Sinovac/Butantan,2021-05-17,2ª Dose
1,Vacina Covid-19 - Covishield,2021-07-16,1ª Dose
2,Vacina Covid-19 - Covishield,2021-06-17,2ª Dose
3,Covid-19-Coronavac-Sinovac/Butantan,2021-03-30,1ª Dose
4,Covid-19-Coronavac-Sinovac/Butantan,2021-04-24,2ª Dose


### Renomeando descrição das vacinas

In [23]:
df = df.replace(to_replace ="Vacina Covid-19 - Covishield", value ="AstraZeneca")
df = df.replace(to_replace ="Covid-19-AstraZeneca", value ="AstraZeneca")
df = df.replace(to_replace ="Covid-19-Coronavac-Sinovac/Butantan", value ="Coronavac")
df = df.replace(to_replace ="Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag", value ="Janssen")
df = df.replace(to_replace ="Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer", value ="Pfizer")
df.sort_values("vacina_descricao_dose", ascending=False)

,vacina_nome,vacina_dataaplicacao,vacina_descricao_dose
506388,Janssen,2021-07-09,Dose
2012349,Janssen,2021-06-29,Dose
2012345,Janssen,2021-06-26,Dose
2012343,Janssen,2021-07-09,Dose
2012341,Janssen,2021-06-27,Dose
...,...,...,...
1398377,AstraZeneca,2021-07-19,1ª Dose
1398376,Pfizer,2021-06-26,1ª Dose
1398375,AstraZeneca,2021-07-20,1ª Dose
1398374,AstraZeneca,2021-06-03,1ª Dose


### Converter unicode "\ xa0" em espaço

Solução para  isso, o que resolveu o problema erro de filtro (1ª dose, 2ª dose):
https://stackoverflow.com/questions/10993612/how-to-remove-xa0-from-string-in-python

In [24]:
df['vacina_descricao_dose'] = df['vacina_descricao_dose'].str.replace('\xa0', ' ')

# ou filtro = df.query("vacina_descricao_dose=='1ª\xa0Dose'") # caso não fosse usado a primeira opção

### Agrupar por vacinas

In [25]:
dados_vacinas = df.groupby(["vacina_nome"]).count()
dados_vacinas.head(5)

,vacina_dataaplicacao,vacina_descricao_dose
vacina_nome,,
AstraZeneca,1267201,1267201
Coronavac,789879,789879
Janssen,54780,54780
Pfizer,635350,635350


### Verficando o quantitativo por dose

In [26]:
doses = df.groupby(["vacina_descricao_dose"]).count()
display(doses)

,vacina_nome,vacina_dataaplicacao
vacina_descricao_dose,,
1ª Dose,1858955,1858955
1ª Dose Revacinação,4,4
2ª Dose,833385,833385
2ª Dose Revacinação,87,87
Dose,54779,54779


### Filtrando dados apenadas da 1ª dose

In [27]:
dose1 = df.query("vacina_descricao_dose=='1ª Dose'")
# ou df.loc[df['vacina_descricao_dose'].str.strip() == '1ª Dose']
dose1.head()

,vacina_nome,vacina_dataaplicacao,vacina_descricao_dose
1,AstraZeneca,2021-07-16,1ª Dose
3,Coronavac,2021-03-30,1ª Dose
5,Coronavac,2021-03-22,1ª Dose
7,Pfizer,2021-07-06,1ª Dose
9,Pfizer,2021-06-12,1ª Dose


### Filtrando dados apenas da 2ª dose ou dose única

In [28]:
dose_jassen = df.loc[df['vacina_descricao_dose'].str.strip() == 'Dose']
dose_2 = df.loc[df['vacina_descricao_dose'].str.strip() == '2ª Dose']

In [29]:
dose2 = pd.merge(dose_jassen, dose_2, how = 'outer')
dose2

,vacina_nome,vacina_dataaplicacao,vacina_descricao_dose
0,Janssen,2021-09-02,Dose
1,Janssen,2021-09-02,Dose
2,Janssen,2021-09-02,Dose
3,Janssen,2021-09-02,Dose
4,Janssen,2021-09-02,Dose
...,...,...,...
888159,AstraZeneca,2021-03-09,2ª Dose
888160,Coronavac,2021-02-09,2ª Dose
888161,AstraZeneca,2021-02-13,2ª Dose
888162,AstraZeneca,2021-02-02,2ª Dose


### Gerando gráficos

In [ ]:
import plotly.express as px
grafico_1 = px.histogram(dose1, x = "vacina_dataaplicacao", color = "vacina_nome",
                         color_discrete_map={
                             "AstraZeneca": "#00CC96",
                             "Coronavac": "#EF553B",
                             "Pfizer": "#AB63FA"},
                         title="Aplicação diária da 1ª dose, em Alagoas",
                         hover_name="vacina_nome", hover_data=["vacina_descricao_dose"])

grafico_1.show()

In [ ]:
grafico_2 = px.histogram(dose2, x = "vacina_dataaplicacao", color = "vacina_nome",
                         color_discrete_map={
                             "AstraZeneca": "#00CC96",
                             "Coronavac": "#EF553B",
                             "Pfizer": "#AB63FA",   
                             "Janssen": "#F9C023"},
                      title = "Aplicação diária da 2ª dose ou dose única, em Alagoas")
grafico_2.show()

### Salvando gráfico em html

In [27]:
grafico_1.write_html("grafico_doses_1.html")
grafico_2.write_html("grafico_doses_2.html")

### Exportando dados da 1ª e 2ª doses

In [30]:
dose1.head()

,vacina_nome,vacina_dataaplicacao,vacina_descricao_dose
1,AstraZeneca,2021-07-16,1ª Dose
3,Coronavac,2021-03-30,1ª Dose
5,Coronavac,2021-03-22,1ª Dose
7,Pfizer,2021-07-06,1ª Dose
9,Pfizer,2021-06-12,1ª Dose


In [31]:
dose1_por_dia = dose1[[
    "vacina_dataaplicacao",
    "vacina_nome",
    "vacina_descricao_dose",]].groupby([
    "vacina_dataaplicacao",
    "vacina_nome"]).count()

In [15]:
dose1_por_dia.to_csv("dose1_por_dia.csv")

In [32]:
dose2.head()

,vacina_nome,vacina_dataaplicacao,vacina_descricao_dose
0,Janssen,2021-09-02,Dose
1,Janssen,2021-09-02,Dose
2,Janssen,2021-09-02,Dose
3,Janssen,2021-09-02,Dose
4,Janssen,2021-09-02,Dose


In [33]:
dose2_por_dia = dose2[[
    "vacina_dataaplicacao",
    "vacina_nome",
    "vacina_descricao_dose",]].groupby([
    "vacina_nome",
    "vacina_dataaplicacao"]).count()

dose2_por_dia

vacina_descricao_dose
vacina_nome vacina_dataaplicacao                       
AstraZeneca 2021-01-18                                1
            2021-01-19                                1
            2021-01-29                                5
            2021-02-01                                1
            2021-02-02                                1
...                                                 ...
Pfizer      2021-09-02                             4880
            2021-09-03                             5325
            2021-09-04                             3442
            2021-09-05                             1217
            2021-09-06                             2165

[608 rows x 1 columns]

In [18]:
dose2_por_dia.to_csv("dose2_por_dia.csv")

### Agrupamento por dia

In [34]:
dose1_data = dose1.groupby(["vacina_dataaplicacao"]).count()
dose1_data.sort_values("vacina_dataaplicacao", ascending=True)

,vacina_nome,vacina_descricao_dose
vacina_dataaplicacao,,
2021-01-17,1,1
2021-01-18,10,10
2021-01-19,482,482
2021-01-20,2101,2101
2021-01-21,3963,3963
...,...,...
2021-09-03,7431,7431
2021-09-04,6600,6600
2021-09-05,2723,2723
